In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
gpu = pd.read_csv('../part_gpu230920.csv')
gpu_bench = pd.read_csv('../benchmark_gpu.csv')

In [3]:
gpu_bench

,name,count
0,GeForce RTX 4090,38968
1,GeForce RTX 4080,34841
2,Radeon PRO W7800,32146
3,GeForce RTX 4070 Ti,31652
4,Radeon RX 7900 XTX,31173
...,...,...
2550,VIA Chrome9 HC3 IGP WDDM 1.1,1
2551,Intel 82852/82855 GM/GME Controller,1
2552,RAGE 128 PRO AGP 4X TMDS,1
2553,Quadro2 Pro,1


In [4]:
gpu.loc[gpu['name'].str.contains('G550'), 'name']

1173    매트록스 밀레니엄 G550 LP PCI 32MB 코트록스
Name: name, dtype: object

In [5]:
gpu_bench.loc[gpu_bench['name'].str.contains('550 '), 'name']

384         T550 Laptop GPU
892      GeForce GTX 550 Ti
2246    Radeon X1550 64-bit
2333    Radeon 9550 / X1050
Name: name, dtype: object

In [6]:
gpu.loc[1173, 'link']

'https://prod.danawa.com/info/?pcode=151759&cate=112753'

In [7]:
unique_values = set()

for row in gpu_bench.name.str.split(r' |-'):
    for value in row:
        unique_values.add(value)

In [8]:
unique_values = set()

for row in gpu.name.str.split(r' |-'):
    for value in row:
        unique_values.add(value)

In [9]:
gpu_regex_map = {'지포스': 'GeForce', '라데온': 'Radeon', '타이탄': 'TITAN',
                 '쿼드로': 'Quadro', 'DUAL': ' ', 'GT1': 'GT 1', 'GT6': 'GT 6',
                 'GT7': 'GT 7', 'GTX': 'GTX ', 'Pro': 'PRO', 'WX': 'WX ', 'W6800X': 'W6800',
                 'W6900X': 'RX 6900'}
bench_regex_map = {'Titan': 'TITAN', '\+': '', 'Quadro': 'Quadro ', 'Pro': 'PRO'}
none_regex_map = {r' +': ' '}

In [10]:
gpu_reg = gpu.replace(regex=gpu_regex_map).replace(regex=none_regex_map)
ben_reg = gpu_bench.replace(regex=bench_regex_map).replace(regex=none_regex_map)

In [11]:
g_names = gpu_reg.name.str.split(r' |-')
bench_names = ben_reg.name.str.split(r' |-')

In [12]:
g_names

0       [이엠텍, GeForce, RTX, 4070, Ti, GAMINGPRO, D6X, ...
1       [이엠텍, GeForce, RTX, 4060, Ti, STORM, X, Dual, ...
2       [갤럭시, GALAX, GeForce, RTX, 4070, EX, GAMER, WH...
3       [이엠텍, GeForce, RTX, 3060, STORM, X, Dual, OC, ...
4        [SAPPHIRE, Radeon, RX, 7600, PULSE, OC, D6, 8GB]
                              ...                        
1316        [액슬, GeForce, GTX, 750, Ti, 클래식, V2, D5, 2GB]
1317               [Radeon, RX, 560, 14CU, D5, 2GB, (중고)]
1318                                 [EVGA, 파워링크, (해외구매)]
1319             [FORSA, GeForce, GT, 610, Plus, D3, 1GB]
1320    [매트록스, M9148, LP, PCle, x16, 1GB, DP/DVI, Quad...
Name: name, Length: 1321, dtype: object

In [13]:
bench_names

0                           [GeForce, RTX, 4090]
1                           [GeForce, RTX, 4080]
2                           [Radeon, PRO, W7800]
3                       [GeForce, RTX, 4070, Ti]
4                        [Radeon, RX, 7900, XTX]
                          ...                   
2550         [VIA, Chrome9, HC3, IGP, WDDM, 1.1]
2551    [Intel, 82852/82855, GM/GME, Controller]
2552             [RAGE, 128, PRO, AGP, 4X, TMDS]
2553                            [Quadro, 2, PRO]
2554                         [FireMV, 2200, PCI]
Name: name, Length: 2555, dtype: object

In [14]:
ignore_text = ['브릿지', 'Sync', '파워링크', 'Box', 'BOX', 'Hercules', '링크패널', 'G550']

In [15]:
bench_list = [0] * len(g_names)
name_list = [''] * len(g_names)
for i in range(len(g_names)):
    gn = g_names.loc[i]
    if any([x in gn for x in ignore_text]):
        bench_list[i] = 0
        name_list[i] = ''
        continue
    max_val = 0
    for j in range(len(bench_names)):
        bn = bench_names.loc[j]
        if 'Laptop' in bn: continue
        res = len(set(gn) & set(bn))
        val = res / float(len(set(gn) | set(bn))) * 100
        if val >= max_val:
            max_val = val
            bench_list[i] = gpu_bench.loc[j, 'count']
            name_list[i] = gpu_bench.loc[j, 'name']
    percent = i / len(g_names) * 100
    print(f'\rgpu: {i+1}/{len(g_names)} ({percent:.0f}%)', end='')

gpu: 1321/1321 (100%)

In [16]:
for g_name, name, bench in zip(gpu.name, name_list, bench_list):
    print(f':{bench:5d}::{name:25s}::{g_name}:')

:31652::GeForce RTX 4070 Ti      ::이엠텍 지포스 RTX 4070 Ti GAMINGPRO D6X 12GB:
:22560::GeForce RTX 4060 Ti      ::이엠텍 지포스 RTX 4060 Ti STORM X Dual OC D6 8GB:
:26902::GeForce RTX 4070         ::갤럭시 GALAX 지포스 RTX 4070 EX GAMER WHITE OC D6X 12GB:
:17144::GeForce RTX 3060         ::이엠텍 지포스 RTX 3060 STORM X Dual OC D6 12GB:
:16661::Radeon RX 7600           ::SAPPHIRE 라데온 RX 7600 PULSE OC D6 8GB:
:22560::GeForce RTX 4060 Ti      ::ZOTAC GAMING 지포스 RTX 4060 Ti TWIN Edge D6 8GB:
:12793::GeForce GTX 1660 SUPER   ::이엠텍 지포스 GTX 1660 SUPER MIRACLE II D6 6GB:
:19329::GeForce RTX 4060         ::ASUS DUAL 지포스 RTX 4060 O8G OC D6 8GB:
:19329::GeForce RTX 4060         ::이엠텍 지포스 RTX 4060 STORM X Dual OC D6 8GB:
:20590::GeForce RTX 3060 Ti      ::GIGABYTE 지포스 RTX 3060 Ti Gaming OC D6X 8GB 피씨디렉트:
:38968::GeForce RTX 4090         ::이엠텍 지포스 RTX 4090 GAMEROCK OC D6X 24GB:
:31652::GeForce RTX 4070 Ti      ::ASUS ProArt 지포스 RTX 4070 Ti O12G OC D6X 12GB:
:26902::GeForce RTX 4070         ::MSI 지포스 RTX 4070 게이밍 X 트리오 

In [17]:
gpu['bench_mark'] = bench_list

In [18]:
today = time.strftime('%y%m%d')
pd.DataFrame(gpu).to_csv(f'../part_gpu_wb{today}.csv', index=None)

In [19]:
gpu_bench.loc[gpu_bench['count'] == 25588, 'name']

Series([], Name: name, dtype: object)

In [20]:
gpu

,name,price,link,company,product_seq,image,chipset_company,chipset,nm,base_clock,...,cooling_type,pan_number,length,thickness,pin,feature,as_years,bench_mark,reg_date,bookmark
0,이엠텍 지포스 RTX 4070 Ti GAMINGPRO D6X 12GB,1128990.0,https://prod.danawa.com/info/?pcode=18668606&c...,이엠텍,18668606,//img.danawa.com/prod_img/500000/606/668/img/1...,NVIDIA,RTX 4070 Ti,4.0,2310.0,...,11,3.0,328.9,63.40,100,7,3.0,31652,202301,NaN
1,이엠텍 지포스 RTX 4060 Ti STORM X Dual OC D6 8GB,565530.0,https://prod.danawa.com/info/?pcode=20274764&c...,이엠텍,20274764,//img.danawa.com/prod_img/500000/764/274/img/2...,NVIDIA,RTX 4060 Ti,4.0,2310.0,...,9,2.0,249.9,40.10,10000,7,3.0,22560,202305,NaN
2,갤럭시 GALAX 지포스 RTX 4070 EX GAMER WHITE OC D6X 12GB,844610.0,https://prod.danawa.com/info/?pcode=19773293&c...,갤럭시,19773293,//img.danawa.com/prod_img/500000/293/773/img/1...,NVIDIA,RTX 4070,NaN,1920.0,...,11,3.0,323.0,48.00,10000,5,3.0,26902,202304,NaN
3,이엠텍 지포스 RTX 3060 STORM X Dual OC D6 12GB,383300.0,https://prod.danawa.com/info/?pcode=13463144&c...,이엠텍,13463144,//img.danawa.com/prod_img/500000/144/463/img/1...,NVIDIA,RTX 3060,8.0,NaN,...,11,2.0,245.0,40.00,10000,7,3.0,17144,202102,NaN
4,SAPPHIRE 라데온 RX 7600 PULSE OC D6 8GB,376190.0,https://prod.danawa.com/info/?pcode=20315144&c...,SAPPHIRE,20315144,//img.danawa.com/prod_img/500000/144/315/img/2...,AMD(ATi),RX 7600,NaN,NaN,...,11,2.0,240.0,44.07,10000,1,3.0,16661,202305,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316,액슬 지포스 GTX750 Ti 클래식 V2 D5 2GB,143000.0,https://prod.danawa.com/info/?pcode=7424305&ca...,액슬,7424305,//img.danawa.com/prod_img/500000/305/424/img/7...,NVIDIA,GTX750 Ti,28.0,1020.0,...,9,1.0,162.0,35.00,0,0,NaN,3901,201903,NaN
1317,라데온 RX 560 14CU D5 2GB (중고),NaN,https://prod.danawa.com/info/?pcode=6643450&ca...,NaN,6643450,//img.danawa.com/prod_img/500000/450/643/img/6...,AMD(ATi),해당없음,NaN,NaN,...,0,NaN,NaN,NaN,0,0,NaN,3641,201811,NaN
1318,EVGA 파워링크 (해외구매),22700.0,https://prod.danawa.com/info/?pcode=5406903&ca...,EVGA,5406903,//img.danawa.com/prod_img/500000/903/406/img/5...,NaN,해당없음,NaN,NaN,...,0,NaN,NaN,NaN,0,0,NaN,0,201708,NaN
1319,FORSA 지포스 GT610 Plus D3 1GB,48030.0,https://prod.danawa.com/info/?pcode=3928707&ca...,FORSA,3928707,//img.danawa.com/prod_img/500000/707/928/img/3...,NVIDIA,GT610,40.0,810.0,...,9,1.0,150.0,NaN,0,0,NaN,304,201603,NaN
